## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Chokurdakh,RU,70.63,147.92,54.00,52,55,8.21,broken clouds
1,Freetown,SL,8.48,-13.23,77.23,86,100,6.73,light rain
2,Leningradskiy,RU,69.38,178.42,34.93,93,100,14.92,overcast clouds
3,Bredasdorp,ZA,-34.53,20.04,40.32,93,0,3.44,clear sky
4,Ponta Delgada,PT,37.73,-25.67,72.32,88,20,13.80,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("Please input the minimum temperature you would like for your trip: "))
max_temp = float(input("Please input the maximum temperature you would like for your trip: "))

Please input the minimum temperature you would like for your trip: 70
Please input the maximum temperature you would like for your trip: 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,Freetown,SL,8.48,-13.23,77.23,86,100,6.73,light rain
4,Ponta Delgada,PT,37.73,-25.67,72.32,88,20,13.80,few clouds
6,Dakar,SN,14.69,-17.44,82.53,78,75,6.91,broken clouds
7,La Libertad,EC,-2.23,-80.90,73.06,78,75,10.36,broken clouds
8,Kahului,US,20.89,-156.47,85.96,76,40,20.71,scattered clouds
11,Jamestown,US,42.10,-79.24,80.19,82,90,5.99,overcast clouds
15,Cayenne,GF,4.93,-52.33,75.24,91,75,1.14,light rain
18,Rikitea,PF,-23.12,-134.97,70.56,66,23,14.00,few clouds
20,Puerto Ayora,EC,-0.74,-90.35,73.36,93,85,7.00,overcast clouds
26,Georgetown,MY,5.41,100.34,80.89,85,40,3.44,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 324 entries, 1 to 705
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 324 non-null    object 
 1   Country              324 non-null    object 
 2   Lat                  324 non-null    float64
 3   Lng                  324 non-null    float64
 4   Max Temp             324 non-null    float64
 5   Humidity             324 non-null    int64  
 6   Cloudiness           324 non-null    int64  
 7   Wind Speed           324 non-null    float64
 8   Current Description  324 non-null    object 
dtypes: float64(4), int64(2), object(3)
memory usage: 25.3+ KB


In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Freetown,SL,77.23,light rain,8.48,-13.23,
4,Ponta Delgada,PT,72.32,few clouds,37.73,-25.67,
6,Dakar,SN,82.53,broken clouds,14.69,-17.44,
7,La Libertad,EC,73.06,broken clouds,-2.23,-80.90,
8,Kahului,US,85.96,scattered clouds,20.89,-156.47,
11,Jamestown,US,80.19,overcast clouds,42.10,-79.24,
15,Cayenne,GF,75.24,light rain,4.93,-52.33,
18,Rikitea,PF,70.56,few clouds,-23.12,-134.97,
20,Puerto Ayora,EC,73.36,overcast clouds,-0.74,-90.35,
26,Georgetown,MY,80.89,scattered clouds,5.41,100.34,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")

print("Hotel Search Completed")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel Search Completed


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Freetown,SL,77.23,light rain,8.48,-13.23,The Country Lodge Hotel
4,Ponta Delgada,PT,72.32,few clouds,37.73,-25.67,Vila Nova Hotel
6,Dakar,SN,82.53,broken clouds,14.69,-17.44,Redface Studios Meublés
7,La Libertad,EC,73.06,broken clouds,-2.23,-80.90,Playa Canela Hotel Boutique
8,Kahului,US,85.96,scattered clouds,20.89,-156.47,Maui Seaside Hotel
...,...,...,...,...,...,...,...
694,Baker City,US,85.59,clear sky,44.77,-117.83,Bridge Street Inn
695,Aden,YE,85.55,broken clouds,12.78,45.04,Sama Emirate Hotel
697,Paragominas,BR,80.06,broken clouds,-2.97,-47.48,INOVA
699,Salinopolis,BR,79.23,moderate rain,-0.61,-47.36,Salinas Park Resort Maçarico AP101


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))